# Init & Login

In [1]:
from math import ceil
from random import randrange
from time import sleep

import spotipy
import spotipy.util as util
from IPython.display import clear_output

## Client Info ##
CLIENT_ID     = ""
CLIENT_SECRET = ""
CLIENT_SCOPE  = "user-follow-modify playlist-modify-private playlist-modify-public"
USER_NAME     = "31ytgsr7wdmiaroy77msqpiupdsi"
REDIR_URI     = "https://github.com/jwatson-CO-edu/yt_shuffle_so_good"
AUTH_URL      = 'https://accounts.spotify.com/api/token'
BASE_URL      = 'https://api.spotify.com/v1/'
## API Info ##
_RESPONSE_LIMIT = 100

with open( "../keys/spot_ID.txt" , 'r' ) as f:
    CLIENT_ID = f.readlines()[0].strip()

with open( "../keys/spot_SECRET.txt" , 'r' ) as f:
    CLIENT_SECRET = f.readlines()[0].strip()

token = None
token = util.prompt_for_user_token(
    username      = USER_NAME,
    scope         = CLIENT_SCOPE,
    client_id     = CLIENT_ID,
    client_secret = CLIENT_SECRET,
    redirect_uri  = REDIR_URI
)

print( token )

spot = spotipy.Spotify( auth = token )
clear_output( wait = True )
sleep( 2 )
print( "TOKEN OBTAINED" )

TOKEN OBTAINED


# Playlist Functions

In [2]:

def get_playlist_length( playlist_ID ):
    """ Get the number of total tracks in the playlist """
    response = spot.user_playlist_tracks(
        CLIENT_ID, 
        playlist_ID, 
        fields = 'items,uri,name,id,total', 
        limit  = _RESPONSE_LIMIT
    )
    return response['total']


def reorder_playlist_by_chunks( playlist_ID, chunkSize, Nmoves = None, verbose = False ):
    """ Shuffle the playlist a chunk at a time """
    Ntracks = get_playlist_length( playlist_ID )
    Nchunks = int( ceil( Ntracks / chunkSize ) )
    if Nmoves is None:
        Nmoves = int( Ntracks / chunkSize )
    for i in range( Nmoves ):
        bgnDex   = randrange(0,Nchunks) * chunkSize
        moveSize = min( chunkSize, Ntracks - bgnDex - 1 )
        sendDex  = randrange(0,Ntracks-1)
        try:
            response = spot.playlist_reorder_items(
                playlist_ID, 
                bgnDex, 
                sendDex, 
                range_length = chunkSize
            )
            if verbose:
                print( f"Iteration {i+1}, {bgnDex}-{bgnDex+moveSize} --> {sendDex}:\n\t{response}" )
        except spotipy.SpotifyException as ex:
            if verbose:
                print( f"ERROR, {bgnDex}-{bgnDex+moveSize} --> {sendDex}:\n\t{ex}" )


def complete_reorder_entire_playlist( playlist_ID, verbose = False, phasePause_s = 5 ):
    """ Completely shuffle the playlist in decreasing chunks """
    Ntracks = get_playlist_length( playlist_ID )
    ## Phase 1: Chunks of 100 ##
    if verbose:
        print("\nPhase 1: Chunks of 100\n")
    reorder_playlist_by_chunks( playlist_ID, 100, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 2: Chunks of 10 ##
    if verbose:
        print("\nPhase 2: Chunks of 10\n")
    reorder_playlist_by_chunks( playlist_ID, 10, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 3: Chunks of 5, 1/2 ##
    if verbose:
        print("\nPhase 3: Chunks of 5, 1/2\n")
    Nmoves = int( ceil( Ntracks / 10 ) )
    reorder_playlist_by_chunks( playlist_ID, 5, Nmoves, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 3: 1 at a Time, 1/3 ##
    if verbose:
        print("\nPhase 3: 1 at a Time, 1/3\n")
    Nmoves = int( ceil( Ntracks / 3 ) )
    reorder_playlist_by_chunks( playlist_ID, 1, Nmoves, verbose = verbose )

# Shuffle Playlists!

In [3]:
playlist = {
    'study01' : "0a2qoe6S7lYeZ6nlhZdA0v",
    'study02' : "6gbtR2cBq5PvkghidCvvGk",
    'study03' : "3o3lN2qntdEV7UKTuuC77K",
    'study04' : "41sFSisljvBDMBXtpp5NIw",
    'study05' : "02iS5AFGp8YVuUUqcQf8ys",
    'study06' : "6KI7A4MWrSM7EyKRUjxIi1",
    'study07' : "3V055Md2JdrUT8tX0af7di",
    'study08' : "0tspdJlwSgiyf2O9PO6QaP",
    'study09' : "5mHRBFoQtYy2izeZ66pG95",
    'study10' : "3832xeKGEOAXFJqE4K8kIq",
    'study11' : "65MXR4dubPL9t0P4dgTWvn",
    'study12' : "0ecSAfnD4CulIVnLt26ukI",
    'study13' : "7K9ucByFRgDuZk8KMHeJkL",
    'study14' : "0v26bHydUxcGC5EbMlkjzG",
    'cringe'  : "2AAUYlKM1nXKHkZUpUSFbv",
}

complete_reorder_entire_playlist( playlist['study14'], verbose = True, phasePause_s = 5 )


Phase 1: Chunks of 100



HTTP Error for PUT to https://api.spotify.com/v1/playlists/0v26bHydUxcGC5EbMlkjzG/tracks with Params: {} returned 400 due to Tracks selected to be reordered are out of bounds


ERROR, 400-400 --> 347:
	http status: 400, code:-1 - https://api.spotify.com/v1/playlists/0v26bHydUxcGC5EbMlkjzG/tracks:
 Tracks selected to be reordered are out of bounds, reason: None
Iteration 2, 100-200 --> 236:
	{'snapshot_id': 'AAADjCTJJBXPKN1loJnegwz8/TE0DUBS'}
Iteration 3, 300-400 --> 54:
	{'snapshot_id': 'AAADje8MIhcPXduOft9Yf+ITei/6iE2o'}
Iteration 4, 100-200 --> 218:
	{'snapshot_id': 'AAADjkkbqHF6D0uVR+q4EaM7vPacbWZZ'}

Phase 2: Chunks of 10



HTTP Error for PUT to https://api.spotify.com/v1/playlists/0v26bHydUxcGC5EbMlkjzG/tracks with Params: {} returned 400 due to Tracks selected to be reordered are out of bounds


Iteration 1, 90-100 --> 242:
	{'snapshot_id': 'AAADjxPDbvB7bO63ox9GbbmfdqXG+xek'}
ERROR, 400-400 --> 285:
	http status: 400, code:-1 - https://api.spotify.com/v1/playlists/0v26bHydUxcGC5EbMlkjzG/tracks:
 Tracks selected to be reordered are out of bounds, reason: None
Iteration 3, 250-260 --> 68:
	{'snapshot_id': 'AAADkMzpfrWVZ5AIYrzyk5Yzp2h1RspN'}
Iteration 4, 130-140 --> 347:
	{'snapshot_id': 'AAADkd/eg4jV4A4MePda3PCyPle9tKGq'}
Iteration 5, 330-340 --> 100:
	{'snapshot_id': 'AAADkr18sCv2IpQuTu3nFkeUpNjJosOx'}
Iteration 6, 140-150 --> 220:
	{'snapshot_id': 'AAADkzQJqDOXcKm7ipat0sftH0vEdut+'}
Iteration 7, 50-60 --> 354:
	{'snapshot_id': 'AAADlDeey23vqILPPpipOMIjPDMKi2VE'}
Iteration 8, 220-230 --> 274:
	{'snapshot_id': 'AAADlZ/U4O0FWoM72hz4QQc1CuxVg3mS'}
Iteration 9, 210-220 --> 111:
	{'snapshot_id': 'AAADlpT6UPjBETKi1a/7v7sffLS7bYBh'}
Iteration 10, 150-160 --> 108:
	{'snapshot_id': 'AAADl6wOkVTbvENRUUsKGUkUxg76jr0F'}
Iteration 11, 80-90 --> 252:
	{'snapshot_id': 'AAADmD5QLkPdGAbtKEol77o

HTTP Error for PUT to https://api.spotify.com/v1/playlists/0v26bHydUxcGC5EbMlkjzG/tracks with Params: {} returned 400 due to Tracks selected to be reordered are out of bounds


Iteration 21, 200-205 --> 162:
	{'snapshot_id': 'AAADyRqRlLv4kDkN8J0h5V6/cyoH6Wr6'}
ERROR, 400-400 --> 173:
	http status: 400, code:-1 - https://api.spotify.com/v1/playlists/0v26bHydUxcGC5EbMlkjzG/tracks:
 Tracks selected to be reordered are out of bounds, reason: None
Iteration 23, 180-185 --> 44:
	{'snapshot_id': 'AAADykFdNpFhSJjthvlENhLVFuAIAxi/'}
Iteration 24, 305-310 --> 333:
	{'snapshot_id': 'AAADy39/iH+0XOLyCinNWZuXyaplUjGH'}
Iteration 25, 385-390 --> 23:
	{'snapshot_id': 'AAADzJ/ku6K95bB8W0XVAmNL5cNELpQ2'}
Iteration 26, 250-255 --> 291:
	{'snapshot_id': 'AAADzUHiWc+6RQfqNcNO8LXuOtuRanc4'}
Iteration 27, 70-75 --> 140:
	{'snapshot_id': 'AAADzlOq30w2t9SNL0pc9tEQF1Aki+/T'}
Iteration 28, 260-265 --> 65:
	{'snapshot_id': 'AAADzyB41NNPnB+/NiK86hEa+ZJMNHJg'}
Iteration 29, 350-355 --> 162:
	{'snapshot_id': 'AAAD0NIKvR3Xwa1Ypw6KdyW8TUEYw1rA'}
Iteration 30, 200-205 --> 275:
	{'snapshot_id': 'AAAD0fxDl5gbJAAg1AxqFPfs5t2n4R69'}
Iteration 31, 360-365 --> 73:
	{'snapshot_id': 'AAAD0ikllEKXecT